In [1]:
"""
Basic usage example of the SQL agent with test tables.
"""

import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from backend.core.schema_manager import SchemaManager
from backend.core.relationship_analyzer import RelationshipAnalyzer
from backend.core.schema_relevance_analyzer import SchemaRelevanceAnalyzer
from backend.core.sql_engine import SQLEngine
from backend.database.connection import get_database_engine
from sqlalchemy import inspect

def main():
    # Initialize components
    schema_manager = SchemaManager()
    relationship_analyzer = RelationshipAnalyzer(schema_manager)
    relevance_analyzer = SchemaRelevanceAnalyzer()
    sql_engine = SQLEngine(schema_manager)
    
    # Get all available tables from the database
    engine = get_database_engine()
    inspector = inspect(engine)
    available_tables = inspector.get_table_names()
    print("\nAvailable Tables:")
    print(available_tables)

    # Load schema for all tables
    for table in available_tables:
        schema_manager.load_schema(table)
        print(f"\n{table} Table Schema:")
        print(schema_manager.get_schema(table))

    # Example 1: Simple SELECT query
    query1 = "SELECT r.receipt_id, c.name as customer_name, i.name as item_name, i.price, r.quantity, r.tip FROM receipts r JOIN customers c ON r.customer_id = c.customer_id JOIN items i ON r.item_id = i.item_id LIMIT 5"
    result1 = sql_engine.execute_query(query1)
    print("\nExample 1 - Simple SELECT:")
    print(result1)

    # Example 2: Query with WHERE clause
    query2 = "SELECT r.receipt_id, c.name as customer_name, i.name as item_name, i.price FROM receipts r JOIN customers c ON r.customer_id = c.customer_id JOIN items i ON r.item_id = i.item_id WHERE i.price > 25"
    result2 = sql_engine.execute_query(query2)
    print("\nExample 2 - Filtered SELECT:")
    print(result2)

    # Example 3: Relevance Analysis
    query = "SELECT c.name as customer_name, i.price FROM receipts r JOIN customers c ON r.customer_id = c.customer_id JOIN items i ON r.item_id = i.item_id WHERE i.price > 25"
    relevance = relevance_analyzer.analyze_query(query)
    print("\nQuery Relevance Analysis:")
    print(f"Query: {query}")
    print(f"Relevant Tables: {relevance}")

    # Example 4: Relationship Analysis
    print("\nDatabase Relationships:")
    for table in available_tables:
        relationships = relationship_analyzer.analyze_relationships(table)
        if relationships:
            print(f"\n{table} table relationships:")
            for rel in relationships:
                print(f"- {rel['description']}")

if __name__ == "__main__":
    main()

Device set to use mps:0



Available Tables:
['customers', 'items', 'receipts']

customers Table Schema:
{'columns': [{'name': 'customer_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'name', 'type': 'VARCHAR(32)', 'nullable': True, 'primary_key': 0}, {'name': 'email', 'type': 'VARCHAR(64)', 'nullable': True, 'primary_key': 0}], 'foreign_keys': []}

items Table Schema:
{'columns': [{'name': 'item_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'name', 'type': 'VARCHAR(32)', 'nullable': True, 'primary_key': 0}, {'name': 'price', 'type': 'FLOAT', 'nullable': True, 'primary_key': 0}], 'foreign_keys': []}

receipts Table Schema:
{'columns': [{'name': 'receipt_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'customer_id', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'item_id', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'quantity', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'tip', 'typ

In [2]:
from backend.core.prompt_generator import DynamicPromptGenerator

def test_prompt_generator(question, prompt_generator):
    print(f"\nTesting Question: {question}")
    print("\nGenerated Prompt:")
    prompt = prompt_generator.generate_prompt(question)
    print(prompt)
    print("\n" + "="*80)

def main():
    # Initialize components
    schema_manager = SchemaManager()
    relevance_analyzer = SchemaRelevanceAnalyzer()
    prompt_generator = DynamicPromptGenerator(schema_manager, relevance_analyzer)
    
    # Get all available tables from the database
    engine = get_database_engine()
    inspector = inspect(engine)
    available_tables = inspector.get_table_names()
    print("\nAvailable Tables:")
    print(available_tables)

    # Load schema for all tables
    for table in available_tables:
        schema_manager.load_schema(table)
        print(f"\n{table} Table Schema:")
        print(schema_manager.get_schema(table))

    # Test different types of questions
    test_questions = [
        "What items did Alice buy?",  # Tests JOIN and WHERE
        "What is the total amount spent by each customer?",  # Tests GROUP BY and aggregation
        "What is the most expensive item?",  # Tests ORDER BY and LIMIT
        "How many items did each customer buy?",  # Tests COUNT and JOIN
        "What is the average tip percentage?",  # Tests calculations
        "Which customer spent the most on tips?"  # Tests complex conditions
    ]
    
    print("\nTesting Prompt Generator:")
    for question in test_questions:
        test_prompt_generator(question, prompt_generator)

if __name__ == "__main__":
    main()

Device set to use mps:0



Available Tables:
['customers', 'items', 'receipts']

customers Table Schema:
{'columns': [{'name': 'customer_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'name', 'type': 'VARCHAR(32)', 'nullable': True, 'primary_key': 0}, {'name': 'email', 'type': 'VARCHAR(64)', 'nullable': True, 'primary_key': 0}], 'foreign_keys': []}

items Table Schema:
{'columns': [{'name': 'item_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'name', 'type': 'VARCHAR(32)', 'nullable': True, 'primary_key': 0}, {'name': 'price', 'type': 'FLOAT', 'nullable': True, 'primary_key': 0}], 'foreign_keys': []}

receipts Table Schema:
{'columns': [{'name': 'receipt_id', 'type': 'INTEGER', 'nullable': False, 'primary_key': 1}, {'name': 'customer_id', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'item_id', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'quantity', 'type': 'INTEGER', 'nullable': True, 'primary_key': 0}, {'name': 'tip', 'typ